In [1]:
print("python")

python


In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [3]:
load_dotenv()

True

In [4]:
key=os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=key, model_name="gpt-4o-mini", temperature=0.7)

C:\Users\Ansh\AppData\Local\Temp\ipykernel_15576\3315515517.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key=key, model_name="gpt-4o-mini", temperature=0.7)


In [6]:
with open("C:\MCQGEN\mcqgenerator1\Response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [7]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repteated as well as challenging and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide to create the quiz. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}


"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [10]:
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repteated as well as challenging and check all the questions to be conforming the text as well.\nMake sure to format your response like RESPONSE_JSON below and use it as a guide to create the quiz. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n\n')

In [11]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Ansh\AppData\Local\Temp\ipykernel_15576\4109068563.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
PATH="C:\\MCQGEN\\mcqgenerator1\\test.txt"

In [18]:
print(PATH)

C:\MCQGEN\mcqgenerator1\test.txt


In [19]:
with open(PATH, "r") as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gray imag

In [21]:
TEXT
NUMBER=5
SUBJECT="Data Science"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON

In [22]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(

        {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)

        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Tu

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Data Science students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{"1": {"no": "1", "mcq": "What is data science primarily focused on?", "options": {"a": "Creating programming code", "b": "Extracting knowledge from large data sets", "c": "Developing hardware solutions", "d": "Conducting theoretical research"}, "correct": "b"}, "2": {"no": "2", "mcq": "Which of the following fields is NOT typically integrated into data science?", "options": {"a": "Natural sciences", "b": "Information technology", "c": "Culinary arts", "d": "Medicine"}, "correct": "

In [24]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1873
Prompt Tokens:1190
Completion Tokens:683
Total Cost:0.0005882999999999999


In [26]:
response

{'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner

In [27]:
quiz=response.get("quiz")

In [28]:
quiz=json.loads(quiz)

In [29]:
quiz

{'1': {'no': '1',
  'mcq': 'What is data science primarily focused on?',
  'options': {'a': 'Creating programming code',
   'b': 'Extracting knowledge from large data sets',
   'c': 'Developing hardware solutions',
   'd': 'Conducting theoretical research'},
  'correct': 'b'},
 '2': {'no': '2',
  'mcq': 'Which of the following fields is NOT typically integrated into data science?',
  'options': {'a': 'Natural sciences',
   'b': 'Information technology',
   'c': 'Culinary arts',
   'd': 'Medicine'},
  'correct': 'c'},
 '3': {'no': '3',
  'mcq': "According to Jim Gray, data science is considered a 'fourth paradigm' of science. What are the other three paradigms mentioned?",
  'options': {'a': 'Empirical, statistical, and computational',
   'b': 'Empirical, theoretical, and computational',
   'c': 'Theoretical, experimental, and technological',
   'd': 'Statistical, empirical, and mathematical'},
  'correct': 'b'},
 '4': {'no': '4',
  'mcq': 'What does a data scientist primarily do?',
  '